In [1]:
# Imports

import os
import ollama
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [8]:
import sys
print(sys.executable)

/Users/robinwarden/Desktop/Projects/job-copilot/job-copilot/bin/python


In [3]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [4]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Say hello to Robin!"}
]

In [5]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [6]:
# Let's just make sure the model is loaded

!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [7]:
response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

Hello! I don't know who Robin is, but I'm happy to meet you! Is there a Robin you'd like to talk about (e.g. from Batman or another context)? Or would you just like some conversation?


In [28]:
# 🏗️ Class to represent a job listing webpage

headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Job_Platform:
    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"

        # Remove unwanted tags
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()

        self.text = soup.body.get_text(separator="\n", strip=True)


In [29]:
# 💬 Build a user prompt from the job listing
def user_prompt_for(job_listing):
    user_prompt = f"You are looking at a job listing titled: {job_listing.title}\n"
    user_prompt += "The contents of this listing are as follows. Please provide a short summary in markdown format. \
If there are job requirements, duties, or application instructions, highlight them:\n\n"
    user_prompt += job_listing.text
    return user_prompt


In [52]:
# 🧠 Send the user/system prompt to OpenAI
def ask_ollama(system_prompt, user_prompt, model=MODEL):
    response = ollama.chat(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]
    )
    return response['message']['content']


In [53]:
# 📂 Load resume from file
with open("resume.txt", "r") as file:
    my_resume = file.read()


In [54]:
print(my_resume[:500])  # Show the first 500 characters to make sure it loaded


 
Robin Warden
Software Engineer | Frontend & API Developer | AI Automation Enthusiast
Lawrenceville, GA       
(470) 406-1714     
Email| LinkedIn | GitHub | Portfolio

+Professional Summary

Creative and AI-native Frontend Developer with 3+ years of experience building scalable web and mobile interfaces using React, TypeScript, HTML/CSS, and modern CSS frameworks including Styled Components, Bootstrap, and Tailwind CSS. SME-level React engineer with a strong focus on component reusability, sca


In [55]:
# 📝 Define the system's behavior
system_prompt = (
    "You are a job assistant. I will give you a job listing, and you must compare it with my resume. "
    "If it's a good match, summarize the job in markdown and say why it's a fit. If it's not, reply: 'Not a match.'\n\n"
    f"My resume:\n{my_resume}"
)



In [56]:
# 🚀 Test the job finder on a real job listing
test_url = "https://remoteok.com/remote-javascript-jobs"  #Replace with the url of the job listing you want to test

# Initialize the page and build the prompt
job_page = Job_Platform(test_url)
user_prompt = user_prompt_for(job_page)

# Call Ollama
summary = ask_ollama(system_prompt, user_prompt)

# Show result
print(summary)


Here's an analysis of the two job postings:

**Job Posting 1: Fullstack Software Engineer at BlackCloak**

* **Salary Range**: $67,500 - $105,000 per year
* **Benefits**: Comprehensive medical, dental, and vision plans, flexible vacation plan, paid parental leave, company equity, home office stipend, and more.
* **Remote Work Arrangement**: 100% remote company within the USA.
* **Job Description**: The Fullstack Software Engineer will be responsible for designing, developing, and deploying web and mobile applications, as well as leading technical teams and mentoring junior engineers.

**Job Posting 2: Fullstack Software Engineer at BlackCloak**

 Wait, it seems like there's an error in the job posting. I'll analyze the first one:

* **Salary Range**: $68k - $105k per year
* **Benefits**: Comprehensive medical, dental, and vision plans, flexible vacation plan, paid parental leave, company equity, home office stipend, and more.
* **Remote Work Arrangement**: 100% remote company within th